In [ ]:
import os
import json
import pretty_midi
from midi_player import MIDIPlayer
from midi_player.stylers import basic
from corpus.tokenizer_v2 import MidiTokenizer

def json_to_midi(notes, output_path):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)

    for note in notes:
        instrument.notes.append(
            pretty_midi.Note(
                velocity=note["velocity"],
                pitch=note["pitch"],
                start=note["onset"],
                end=note["offset"],
            )
        )
    midi.instruments.append(instrument)
    midi.write(output_path)

def json_to_midi_with_beat(notes, output_path, beat_json_path):
    midi = pretty_midi.PrettyMIDI()
    
    instrument = pretty_midi.Instrument(program=0)
    for note in notes:
        instrument.notes.append(
            pretty_midi.Note(
                velocity=note["velocity"],
                pitch=note["pitch"],
                start=note["onset"],
                end=note["offset"],
            )
        )
    midi.instruments.append(instrument)
    
    if not os.path.exists(beat_json_path):
        print(f"Missing beat info JSON file: {beat_json_path}")
        return
    with open(beat_json_path, "r") as f:
        beat_info = json.load(f)
    
    metronome_track = pretty_midi.Instrument(
        program=115, is_drum=True, name="Metronome"
    )
    beat_duration = 0.1 
    for downbeat in beat_info.get("downbeat_pred", []):
        note_downbeat = pretty_midi.Note(
            velocity=100,
            pitch=36,
            start=downbeat,
            end=downbeat + beat_duration,
        )
        metronome_track.notes.append(note_downbeat)
    
    for beat in beat_info.get("beat_pred", []):
        note_beat = pretty_midi.Note(
            velocity=100,
            pitch=38,
            start=beat,
            end=beat + beat_duration,
        )
        metronome_track.notes.append(note_beat)
    
    midi.instruments.append(metronome_track)
    
    midi.write(output_path)


def get_midi_player(midi_file_path):
    return MIDIPlayer(url_or_file=midi_file_path, height=600, styler=basic, title='My Player')

base_folder = "../../dataset/0000/1/"
midi_path = base_folder + "aligned_transcription.json"
tempo_path = base_folder + "tempo.json"
beats_path = base_folder + "beats.json"
midi_file_path = base_folder + "detokenize.mid"

tokenizer = MidiTokenizer(tempo_path)
events = tokenizer.encode(midi_path, with_grace_note=True)
for e in events:
    print(e)

# tokenizer.decode_to_score(events)
# restored_notes = tokenizer.restore()

tokenizer = MidiTokenizer(tempo_path)
decoded_notes = tokenizer.decode_to_notes(events)

json_to_midi(decoded_notes, midi_file_path)
get_midi_player(midi_file_path)

# with open(midi_path, 'r') as f:
#     notes = json.load(f)

# json_to_midi_with_beat(decoded_notes, midi_file_path, beats_path)
# get_midi_player(midi_file_path)

In [ ]:
import json
import pretty_midi
from midi_player import MIDIPlayer
from midi_player.stylers import basic

def json_to_midi(notes, output_path):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)

    for note in notes:
        instrument.notes.append(
            pretty_midi.Note(
                velocity=note["velocity"],
                pitch=note["pitch"],
                start=note["onset"],
                end=note["offset"],
            )
        )
    midi.instruments.append(instrument)
    midi.write(output_path)

def get_midi_player(midi_file_path):
    return MIDIPlayer(url_or_file=midi_file_path, height=600, styler=basic, title='My Player')

out_dir = "./infer/output/"
eval_dir = "./dataset/eval/"
src_dir = "./infer/src/"

extract_path = src_dir + "extract.json"
# origin_path = src_dir + "cover.json"
out_path = out_dir + "output.json"
# midi_file_path = out_dir + "picogen.mid"
midi_file_path = out_dir + "output.mid"
# midi_file_path = eval_dir + "JPOP4/picogen.mid"

with open(extract_path, "r") as f:
    notes = json.load(f)
# with open(origin_path, "r") as f:
#     notes = json.load(f)
with open(out_path, "r") as f:
    notes = json.load(f)

json_to_midi(notes, midi_file_path)

get_midi_player(midi_file_path)

JPOP10 -> [2211]

In [ ]:
from utils.midi_tool import midi_to_wav

dir_name = "JPOP10"
name = "picogen"
name = "amtapc"
name = "music2midi"

input_midi_path = f"./dataset/eval/{dir_name}/{name}.mid"
output_wav_path = f"./dataset/eval/{dir_name}/{name}.wav"
sound_font_path = "./utils/sound_font/SGM-v2.01.sf2"

midi_to_wav(input_midi_path, output_wav_path, sound_font_path)

In [ ]:
import json
import os
import numpy as np
import librosa
import soundfile as sf

# 假設您的 MIDI 轉換工具函式位於此處
from utils.midi_tool import midi_to_wav

def normalize_and_overwrite(audio_path, target_peak=0.98):
    """
    載入指定的 WAV 檔案，將其音量正規化後，覆蓋原始檔案。

    Args:
        audio_path (str): 要處理的音訊路徑。
        target_peak (float): 目標峰值振幅，設為略小於 1.0 可避免削波。
    """
    try:
        Fs = 44100 # 確保取樣率與您的專案一致
        audio, _ = librosa.load(audio_path, sr=Fs)
        
        # 如果音訊振幅已經很正常或幾乎是靜音，則跳過以節省時間
        max_amplitude = np.max(np.abs(audio))
        if max_amplitude > 0.9 or max_amplitude < 1e-6:
            # print(f"  - 音量已在正常範圍，無需正規化: {os.path.basename(audio_path)}")
            return

        # 計算正規化因子並應用
        normalized_audio = audio / max_amplitude * target_peak
        
        # 使用 soundfile 覆蓋原始檔案
        sf.write(audio_path, normalized_audio, Fs)
        # print(f"  - 音量已成功正規化: {os.path.basename(audio_path)}")

    except Exception as e:
        print(f"  ❌ 正規化失敗: {audio_path}，錯誤: {e}")


def convert_and_normalize_all_versions():
    """
    讀取 metadata.json，為指定的 MIDI 版本批次轉換為 WAV 檔案，
    並立即對其進行音量正規化。如果目標 WAV 檔案已存在，則跳過。
    """
    # --- 1. 設定路徑和常數 ---
    base_dir = os.path.join(".", "dataset", "eval")
    metadata_path = os.path.join(base_dir, "metadata.json")
    sound_font_path = os.path.join(".", "utils", "sound_font", "SGM-v2.01.sf2")
    
    VERSION_MAPPING = {
        "picogen": "picogen",
        "amtapc": "amtapc",
        "music2midi": "music2midi",
        "cover": "human"
    }

    # --- 2. 讀取 metadata.json ---
    try:
        with open(metadata_path, 'r', encoding='utf-8') as f:
            metadata = json.load(f)
        print(f"✅ 成功讀取 metadata.json，共找到 {len(metadata)} 首歌曲。")
    except Exception as e:
        print(f"❌ 讀取 metadata.json 失敗: {e}")
        return

    # --- 3. 遍歷歌曲和版本，進行轉換與正規化 ---
    total_processed = 0
    for i, song_data in enumerate(metadata):
        dir_name = song_data.get("dir_name")
        if not dir_name:
            print(f"⚠️ 警告：第 {i+1} 筆資料缺少 'dir_name'，已跳過。")
            continue

        print(f"\n🎵 正在處理歌曲目錄: {dir_name} ({i+1}/{len(metadata)})")

        for input_name, output_name in VERSION_MAPPING.items():
            input_midi_path = os.path.join(base_dir, dir_name, f"{input_name}.mid")
            output_wav_path = os.path.join(base_dir, dir_name, f"{output_name}.wav")

            # --- 【關鍵修改】重新加入檢查，若目標 WAV 檔案已存在，則跳過 ---
            if os.path.exists(output_wav_path):
                print(f"  ↪️ 已跳過 (WAV 已存在): {os.path.basename(output_wav_path)}")
                continue
            
            # 檢查來源 MIDI 是否存在
            if not os.path.exists(input_midi_path):
                # 因為上面已經跳過已存在的檔案，這裡的訊息可以選擇性關閉，避免過多輸出
                # print(f"  ↪️ 已跳過 (MIDI 不存在): {input_midi_path}")
                continue
            
            try:
                # 步驟 A: 執行 MIDI -> WAV 轉換
                print(f"  🔄 正在轉換: {os.path.basename(input_midi_path)} -> {os.path.basename(output_wav_path)}")
                midi_to_wav(input_midi_path, output_wav_path, sound_font_path)
                
                # 步驟 B: 立刻對剛生成的 WAV 檔案進行音量正規化並覆蓋
                normalize_and_overwrite(output_wav_path)
                
                print(f"  ✅ 成功轉換並正規化: {os.path.basename(output_wav_path)}")
                total_processed += 1
            except Exception as e:
                print(f"  ❌ 處理失敗: {input_midi_path}，錯誤: {e}")

    print(f"\n🎉 --- 所有任務已完成！總共處理了 {total_processed} 個新檔案。 ---")


# --- 安裝必要的函式庫 ---
# 如果您尚未安裝 librosa 或 soundfile，請執行:
# pip install librosa soundfile

# --- 執行主程式 ---
if __name__ == '__main__':
    convert_and_normalize_all_versions()

In [ ]:
from midi_player import MIDIPlayer
from midi_player.stylers import basic

midi_file_path = "./dataset/eval/JPOP10/etude_d.mid"

MIDIPlayer(url_or_file=midi_file_path, height=600, styler=basic, title='My Player')

In [ ]:
import json
import os
from corpus import Synchronizer

def batch_process_warping_paths():
    """
    讀取 metadata.json，對每首歌的四個版本與 origin.wav 計算對齊路徑(wp)，
    並利用 Synchronizer 的快取機制自動儲存結果到各目錄的 wp.json 中。
    """
    # --- 1. 設定路徑和常數 ---
    base_dir = os.path.join(".", "dataset", "eval")
    metadata_path = os.path.join(base_dir, "metadata.json")
    origin_filename = "origin.wav"
    
    # 定義要處理的四個翻奏版本名稱
    cover_versions = ["human", "picogen", "amtapc", "music2midi"]

    # --- 2. 讀取 metadata.json ---
    try:
        with open(metadata_path, 'r', encoding='utf-8') as f:
            metadata = json.load(f)
        print(f"✅ 成功讀取 metadata.json，共找到 {len(metadata)} 首歌曲。")
    except FileNotFoundError:
        print(f"❌ 錯誤：找不到 metadata.json 檔案，請確認路徑 '{metadata_path}' 是否正確。")
        return

    # --- 3. 實例化 Synchronizer ---
    # 可以在迴圈外實例化一次，重複使用
    synchronizer = Synchronizer()

    # --- 4. 遍歷每首歌曲和每個版本進行計算 ---
    for i, song_data in enumerate(metadata):
        dir_name = song_data.get("dir_name")
        if not dir_name:
            print(f"⚠️ 警告：第 {i+1} 筆資料缺少 'dir_name'，已跳過。")
            continue

        song_dir = os.path.join(base_dir, dir_name)
        print(f"\n🎵 === 處理歌曲目錄: {song_dir} ({i+1}/{len(metadata)}) ===")

        # 檢查 origin.wav 是否存在
        origin_wav_path = os.path.join(song_dir, origin_filename)
        if not os.path.exists(origin_wav_path):
            print(f"  ❌ 錯誤：找不到基準檔案 {origin_wav_path}，無法進行比較，已跳過此目錄。")
            continue

        # 遍歷需要處理的四個版本
        for version_name in cover_versions:
            print(f"  --- 版本: {version_name} ---")
            cover_wav_path = os.path.join(song_dir, f"{version_name}.wav")

            # 檢查 cover audio 是否存在
            if not os.path.exists(cover_wav_path):
                print(f"  ↪️ 已跳過 (WAV 不存在): {cover_wav_path}")
                continue
            
            try:
                synchronizer.get_wp(origin_wav_path, cover_wav_path, song_dir)
            except Exception as e:
                print(f"  ❌ 處理失敗: {cover_wav_path}，錯誤: {e}")

    print("\n🎉🎉🎉 --- 所有歌曲目錄的對齊路徑(wp)都已處理完成！ ---")


# --- 執行主程式 ---
if __name__ == '__main__':
    batch_process_warping_paths()

https://youtu.be/6Q0Pd53mojY\?si\=dJuemwnwfdlsgbfk
https://youtu.be/Ug5-kXqP5l8\?si\=WwW9_D6QyBSO6cXZ
https://youtu.be/iFIXi6zzCls?si=kYxheOnqR3573IZp
https://youtu.be/kbNdx0yqbZE?si=4Ze8lkq-LGflsvJE
https://youtu.be/OLRbIc8KZ_8?si=nCbfnyRqRdofOudC 
https://youtu.be/s1bZEnGAX8I\?si\=LigiA3P9sxbBNFwj (musicxml error)
https://youtu.be/4MoRLTAJY_0\?si\=QvLpDCztTiz_wWIT (beat detection error)
https://youtu.be/wgwIfD9Ihik?si=Pap3maz0ho4fv16v
https://youtu.be/JQ2913bVo30?si=7TNGouF9baF_iZWg
https://youtu.be/zjEMFuj23B4?si=kmmHvJ4Wh-ariHIn

#### CPOP

- https://youtu.be/OLRbIc8KZ_8?si=HTPDSGHKPtESid2G 2 3 4
- https://youtu.be/in8NNzwFa-s?si=A9BuyurRE4UPfNtJ
- https://youtu.be/HQ_mU73VhEQ?si=z0Qgj89QVm0P6bVl 4 4 4
- https://youtu.be/8MG--WuNW1Y\?si\=6Y38ZiA2l7ZdKtd9 3 4 4
- https://youtu.be/ZPALMaXLfIw?si=MCc7w7vwrDdlDhNm 3 3 3
- https://youtu.be/h0qYPIlE9us?si=mTG3oz-52Ou2bOi7 2 3 2

---
#### JPOP
https://youtu.be/kbNdx0yqbZE?si=B40NV4X87AqOyx5g 2 2 2
https://youtu.be/Yq7e_AY0dnk?si=KZi6YMpaT6CIWs_g 2 3 4
https://youtu.be/M-Eyhjkepy0?si=wztq11Kp4xCHOqjz 2 3 3
https://youtu.be/fp3F6TqBsAU\?si\=QJ_QvqZoH4HXvf2R 2 2 3
https://youtu.be/bVUEuXOjeDc?si=Wq-ujMPc8qWHnwi3
https://youtu.be/XwgL4C2WaU8?si=9yP_3iEI4YGhfUdr